In [1]:
import sys
import os
from keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from skimage.io import imread
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd

In [2]:
K.clear_session()

In [3]:
data_entrenamiento = './train'
data_validacion = './test'

In [4]:
#Parametros red neuronal

epocas = 20
altura, longitud = 100, 100
batch_size = 30
pasos = 210
pasos_validacion = 50
filtrosConv1 = 60
filtrosConv2 = 100
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 29
lr = 0.0005

In [5]:
images_df = pd.read_csv(r'.\\file_target.csv')

images_df = images_df.rename(columns={'archivo': 'filenames',
                       'target': 'category'})

#images_df = images_df.sort_values('category')

images_df['category'].unique()

array(['G', 'H', 'X', 'Q', 'F', 'B', 'V', 'U', 'D', 'Y', 'L', 'space',
       'Z', 'A', 'M', 'I', 'C', 'P', 'J', 'nothing', 'O', 'E', 'S', 'R',
       'del', 'T', 'K', 'W', 'N'], dtype=object)

In [6]:
entrenamiento, validacion = train_test_split(images_df, test_size = 0.19)

entrenamiento = entrenamiento.reset_index(drop=True)

#entrenamiento = entrenamiento.sort_values('category')

validacion = validacion.reset_index(drop=True)

#validacion = validacion.sort_values('category')

In [7]:
entrenamiento

,filenames,category
0,3646.jpg,A
1,1869.jpg,G
2,3650.jpg,Y
3,8826.jpg,R
4,1376.jpg,A
...,...,...
7285,632.jpg,W
7286,1998.jpg,I
7287,459.jpg,space
7288,8802.jpg,K


In [8]:
#Pre-procesamiento de las imágenes
#Primero creamos un generador que dice como vamos a preprocesar la info

entrenamiento_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                          shear_range = 0.3,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

validacion_datagen = ImageDataGenerator(rescale = 1.0/255.0)

imagen_entrenamiento = entrenamiento_datagen.flow_from_dataframe(entrenamiento,
                                                                 data_entrenamiento,
                                                                 target_size=(altura, longitud),
                                                                 x_col='filenames',
                                                                y_col='category',
                                                                batch_size=batch_size,
                                                                class_mode='categorical')

imagen_validacion = validacion_datagen.flow_from_dataframe(validacion,
                                                           data_entrenamiento,
                                                          target_size=(altura, longitud),
                                                           x_col='filenames',
                                                            y_col='category',
                                                          batch_size=batch_size,
                                                          class_mode='categorical')

Found 7290 validated image filenames belonging to 29 classes.
Found 1710 validated image filenames belonging to 29 classes.


In [9]:
cnn=Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding='same', input_shape=(altura, longitud, 3), activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding='same', activation='relu'))

cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())

cnn.add(Dense(256, activation='relu'))

cnn.add(Dropout(0.5))

cnn.add(Dense(clases, activation='softmax'))

In [10]:
cnn.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=lr), metrics=['accuracy'])

In [11]:
modelo='./modelo/modelo1.h5'

cnn = load_model(modelo)

cnn.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion, validation_steps=pasos_validacion)

dir = './modelo/'

if not os.path.exists(dir):
    os.mkdir(dir)
cnn.save('./modelo/modelo1.h5')
cnn.save_weights('./modelo/pesos1.h5')

Epoch 1/20
210/210 [==============================] - 27s 128ms/step - loss: 0.7253 - accuracy: 0.7324 - val_loss: 0.2022 - val_accuracy: 0.9420
Epoch 2/20
210/210 [==============================] - 28s 133ms/step - loss: 0.7315 - accuracy: 0.7343 - val_loss: 0.1947 - val_accuracy: 0.9453
Epoch 3/20
210/210 [==============================] - 28s 134ms/step - loss: 0.7156 - accuracy: 0.7419 - val_loss: 0.2225 - val_accuracy: 0.9393
Epoch 4/20
210/210 [==============================] - 28s 134ms/step - loss: 0.7028 - accuracy: 0.7475 - val_loss: 0.2088 - val_accuracy: 0.9413
Epoch 5/20
210/210 [==============================] - 28s 134ms/step - loss: 0.7383 - accuracy: 0.7359 - val_loss: 0.2073 - val_accuracy: 0.9487
Epoch 6/20
210/210 [==============================] - 28s 134ms/step - loss: 0.6937 - accuracy: 0.7495 - val_loss: 0.1995 - val_accuracy: 0.9447
Epoch 7/20
210/210 [==============================] - 30s 142ms/step - loss: 0.6632 - accuracy: 0.7573 - val_loss: 0.2082 - val_ac